# Trying another method

Based on this tutorial: https://huggingface.co/blog/fine-tune-wav2vec2-english

In [1]:
# Import packages
import os
import sys
import random
import pandas as pd
import torch
import torchaudio
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer, Wav2Vec2Processor, Wav2Vec2ForCTC, TrainingArguments, Trainer
from datasets import Dataset, Audio
from peft import LoraConfig, get_peft_model
import json

/opt/anaconda3/envs/lora_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load training dataset

In [ ]:
# Set random seed for reproducibility
random.seed(1234)
torch.manual_seed(1234)

# Load participant sets CSV
participant_df = pd.read_csv("/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/participant_sets.csv")
training_ids = participant_df[participant_df['set'].str.lower() == 'training']['ParticipantID'].str.lower().tolist()
training_ids = [id.strip().lower() for id in training_ids]
eval_ids = participant_df[participant_df['set'].str.lower() == 'evaluation']['ParticipantID'].str.lower().tolist()

# Load csv of incorrect utterances
# Create set of included files for training
inclusion_df = pd.read_csv("/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/incorrect_utterances.csv")
incorrect_word_files = inclusion_df[inclusion_df['Subject inclusion'].str.lower() == 'include']['Filename'].tolist()
incorrect_word_files = set([f.lower() for f in incorrect_word_files])  # normalize case

# Extract filepath, participant info, and word from KT1 data file directory
base_dir = "/home/cogsci-lasrlab/Documents/CSS_Capstone/KT1_data"
data = []
included_count = 0
excluded_count = 0
num_files = 0
num_train = 0
num_eval = 0

for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith('.wav') and "participant_" in file:
                base = file[:-4]  # remove '.wav'
                before, word = base.split("participant_")
                    
                # Remove 'K1' and researcher number before 'participant_'
                ppt_id = before.replace("K1", "")[:-1]
                ppt_id_clean = ppt_id.strip().lower() 
                
                # handle missing participant
                if "rri0" in file.lower():
                    ppt_id_clean = "rri0"
                num_files += 1

                # Check if file has correct word
                if file.lower() not in incorrect_word_files:
                    included_count += 1
                    # Only include training data
                    if ppt_id_clean.lower() in training_ids:
                        num_train += 1
                        data.append({
                            "file_name": os.path.join(folder_path, file),
                            "ppt_id": ppt_id,
                            "transcription": word
                            })
                                
                    #Count number of excluded file
                    else:
                        num_eval += 1
                else:
                    excluded_count += 1

# Convert to DataFrame
df = pd.DataFrame(data)

# Report
print(f"{num_files} total files.") # should be 1728
print(f"{included_count} files were included (both training and evaluation).") # should be 1591
print(f"{num_train} files are used for training.") # should be 1192
print(f"{num_eval} files are used fr evaluation.") # should be 399
print(f"{excluded_count} files were excluded due to incorrect word.") # should be 137

# Convert to dataset
dataset = Dataset.from_pandas(df)

# Clean structure for downstram preprocessing
dataset = dataset.rename_column("transcription", "text") 
dataset = dataset.rename_column("file_name", "audio") 
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# Train-test split
dataset = dataset.train_test_split(test_size=0.2)

1728 total files.
1591 files were included (both training and evaluation).
1192 files are used for training.
399 files are used fr evaluation.
137 files were excluded due to incorrect word.


## Create custom tokenizer

In [29]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names["train"])

vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Map: 100%|██████████| 298/298 [00:00<00:00, 57841.76 examples/s]


## Initialize feature extractor

In [32]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

## Wrap feature extractor and tokenizer

In [51]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

## Preprocess data

In [34]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    batch["labels"] = processor(text=batch["text"]).input_ids

    return batch


In [35]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=1)


Map: 100%|██████████| 239/239 [00:00<00:00, 427.31 examples/s]


## Train

### Initialize CTC Data Collator

In [36]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [37]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

### Try fine-tuning wav2vec (without LoRA)

In [ ]:
# Instantiate Wav2Vec2 model
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

# Freeze feature extractor layer
model.freeze_feature_extractor()

/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/.venv/lib/python3.12/site-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/.venv/lib/python3.12/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2175: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze

In [ ]:
# Initialize training arguments
training_args = TrainingArguments(
  output_dir="/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/finetuned_wav2vec2",
  group_by_length=True,
  per_device_train_batch_size=5,
  eval_strategy="no",
  num_train_epochs=1,
  fp16=False,
  gradient_checkpointing=True, 
  save_steps=500,
  eval_steps=500,
  logging_steps=10,
  learning_rate=1e-4,
  weight_decay=0.0,
  warmup_steps=1000,
  save_total_limit=2,
    label_names=['labels']
)

In [ ]:
# Instantiate Trainer
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor
)

/tmp/ipykernel_209626/549408762.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Training loop
trainer.train()

/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/.venv/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss
10,44.225000
20,32.058300
30,27.662000
40,23.076000
50,27.469500
60,33.985400
70,19.968200
80,11.789600
90,6.410800
100,6.212500


TrainOutput(global_step=191, training_loss=14.087261709243215, metrics={'train_runtime': 453.5991, 'train_samples_per_second': 2.101, 'train_steps_per_second': 0.421, 'total_flos': 7005069726364800.0, 'train_loss': 14.087261709243215, 'epoch': 1.0})

### Inject LoRA into Wav2Vec2 Base Model

In [148]:
# initialize base model
# base_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
base_model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base-960h", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id
)

# Configure LoRA arguments
lora_config = LoraConfig(
    r=8, # can change rank, r=8 is most common
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "v_proj"] # attention layers
)

# Add LoRA layers to Wav2Vec2 model
model = get_peft_model(base_model, lora_config)

# Do not train feature extractor
model.freeze_feature_extractor()

# Print number and proportion of trainable parameters
model.print_trainable_parameters()


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 294,912 || all params: 94,691,232 || trainable%: 0.3114


/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/.venv/lib/python3.12/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2175: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


### Initialize Training Arguments

In [149]:
# training_args = TrainingArguments(
#     output_dir="/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/lora_test", # use absolute path
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=1,
#     eval_strategy="no",
#     save_strategy="no",
#     learning_rate=5e-4,
#     weight_decay=0.01, # L2 regularization, lower if the loss 
#     num_train_epochs=1, # make this larger
#     fp16=False,
#     gradient_accumulation_steps=5, # experiment with making this larger (larger = faster)
#     # logging_dir="./logs_test",
#     logging_steps=10,
#     report_to="none",
#     label_names=["labels"]
# )

training_args = TrainingArguments(
  output_dir="/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/lora_test",
  group_by_length=True,
  per_device_train_batch_size=5,
  eval_strategy="no",
  num_train_epochs=1,
  fp16=False,
  gradient_checkpointing=True, 
  save_steps=500,
  eval_steps=500,
  logging_steps=10,
  learning_rate=1e-4,
  weight_decay=0.0,
  warmup_steps=1000,
  save_total_limit=2,
    label_names=['labels']
)

In [150]:
class SkipBatchTrainer(Trainer):
    def train_dataloader(self):
        dataloader = super().get_train_dataloader()
        return self.__skip_error_dataloader()
    
    def skip_error_dataloader(self, dataloader):
        for batch in dataloader:
            try:
                yield batch
            except Exception as e:
                print(f"[WARNING] Skipping batch due to error: {e}")
                continue

    def training_step(self, *args, **kwargs):
        try:
            return super().training_step(*args, **kwargs)
        except Exception as e:
            print(f"[WARNING] Forward pass failed, skipping batch: {e}")
            return torch.tensor(0.0).to(self.args.device)

In [151]:
trainer = SkipBatchTrainer(
    model=model,
    data_collator = data_collator,
    args=training_args,
    # compute_metrics=compute_metrics,
    train_dataset=dataset['train'],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor
)

/tmp/ipykernel_203099/260261800.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SkipBatchTrainer.__init__`. Use `processing_class` instead.
  trainer = SkipBatchTrainer(


### Train

In [152]:
trainer.train()
print("Done training.")
model.save_pretrained("/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/lora_test") # use absolute path
processor.save_pretrained("/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/lora_test") # use absolute path

/home/cogsci-lasrlab/Documents/CSS_Capstone/lora_charsiu/.venv/lib/python3.12/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[WARNING] Forward pass failed, skipping batch: element 0 of tensors does not require grad and does not have a grad_fn


Step,Training Loss


[WARNING] Forward pass failed, skipping batch: element 0 of tensors does not require grad and does not have a grad_fn
[WARNING] Forward pass failed, skipping batch: element 0 of tensors does not require grad and does not have a grad_fn
[WARNING] Forward pass failed, skipping batch: element 0 of tensors does not require grad and does not have a grad_fn
[WARNING] Forward pass failed, skipping batch: element 0 of tensors does not require grad and does not have a grad_fn
[WARNING] Forward pass failed, skipping batch: element 0 of tensors does not require grad and does not have a grad_fn
[WARNING] Forward pass failed, skipping batch: element 0 of tensors does not require grad and does not have a grad_fn
[WARNING] Forward pass failed, skipping batch: element 0 of tensors does not require grad and does not have a grad_fn


: 

## NEXT STEPS

- MAKE `vocab.json` ALL CAPS